In [1]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
from numpy import asarray
from tqdm import tqdm

In [2]:
TRAIN_DIR = '/kaggle/input/cv-2022-project-scene-understanding/Train-20220409T231819Z-001/Train/Images'
data = pd.read_csv('/kaggle/input/cv-2022-project-scene-understanding/TrainImageLabels.csv')
Data=np.array(data)
Test_DIR = '../input/cv-2022-project-scene-understanding/Test/Test/Images'
Test_data = pd.read_csv('/kaggle/input/cv-2022-project-scene-understanding/TestImageLabels.csv')
Test_data=np.array(Test_data)

In [3]:
train_names=Data[:,0]
test_names=Test_data[:,0]
trainlight =Data[:,1]
testlight =Test_data[:,1]
trainlabels= Data[:,2]
testlabels= Test_data[:,2]

In [4]:
def gammaCorrection(src, gamma):
    invGamma = 1 / gamma
 
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
 
    return cv2.LUT(src, table)

In [5]:
def create_data(TRAIN_DIR,names,labels,lowLightMood):
    Xtrain =[]
    Ytrain = []
    for i in tqdm((range(len(names)))):
        path = os.path.join(TRAIN_DIR, names[i])
        imgArr = cv2.imread(path)
        if lowLightMood[i]==2 or lowLightMood[i]==5 or  lowLightMood[i]==3 or  lowLightMood[i]==4 or lowLightMood[i]==9:
            imgArr = gammaCorrection(imgArr, 2)
        elif lowLightMood[i]==6 or lowLightMood[i]==10 or lowLightMood[i]==7 or lowLightMood[i]==8 :
            imgArr = gammaCorrection(imgArr, 1.6)
        elif lowLightMood[i]==1:
            i+=1
            continue
        imgArr = cv2.resize(imgArr, (300, 200))
        imgArr = imgArr.reshape(200, 300, 3)
        Xtrain.append(np.array(imgArr))
        if labels[i]==1:
            Ytrain.append(np.array([1, 0]))
        elif labels[i]==2:
            Ytrain.append(np.array([0, 1]))
        
    return Xtrain , Ytrain

In [6]:
X ,Y = create_data(TRAIN_DIR,train_names,trainlabels,trainlight)
Xtest ,Ytest = create_data(Test_DIR,test_names,testlabels,testlight)

100%|██████████| 1800/1800 [00:49<00:00, 36.41it/s]


In [7]:
Xt = np.array(X)
Yt = np.array(Y)
Xtest = np.array(Xtest)
Ytest = np.array(Ytest)



In [8]:
from keras.applications import vgg16
img_rows, img_cols = 200, 300 


vgg = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3),
                 classes=2)

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    def lw(bottom_model, num_classes):
    #"""creates the top or head of the model that will be placed ontop of the bottom layers"""
        top_model = bottom_model.output
        top_model = GlobalAveragePooling2D()(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(512,activation='relu')(top_model)
        top_model = Dense(num_classes,activation='softmax')(top_model)
        return top_model 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

from keras.models import Model


num_classes = 2

FC_Head = lw(vgg, 2)

model = Model(inputs = vgg.input, outputs = FC_Head)

print(model.summary())


2022-11-01 16:30:00.433890: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 16:30:00.615257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 16:30:00.616056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 16:30:00.617677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step
0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 150, 64)      0        

In [9]:
from tensorflow.keras.models import Model
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(Xt,Yt,epochs=35)



2022-11-01 16:30:05.293497: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/35


2022-11-01 16:30:06.811830: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


93/93 [==============================] - 17s 80ms/step - loss: 0.6835 - accuracy: 0.8440
Epoch 2/35
93/93 [==============================] - 7s 73ms/step - loss: 0.1978 - accuracy: 0.9220
Epoch 3/35
93/93 [==============================] - 7s 73ms/step - loss: 0.1499 - accuracy: 0.9427
Epoch 4/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0956 - accuracy: 0.9624
Epoch 5/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0842 - accuracy: 0.9705
Epoch 6/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0642 - accuracy: 0.9773
Epoch 7/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0591 - accuracy: 0.9759
Epoch 8/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0301 - accuracy: 0.9898
Epoch 9/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0408 - accuracy: 0.9881
Epoch 10/35
93/93 [==============================] - 7s 73ms/step - loss: 0.0321 - accuracy: 0.9905
Epoch 11/35
93/93 [

In [10]:
scores = model.evaluate(Xtest,Ytest, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 91.00%
